В этом ноутбуке мы с вами попробуем применить рекуррентные сети к задаче классификации текста, а также реализовать модуль внимания и имплементировать сеть вместе с ним.

Как обычно, сначала убедимся, что у нас установлены все необходимые пакеты.

In [ ]:
!pip install pandas tensorflow 

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


Далее импортируем необходимые слои, включая технические Layer и backend - K. Данные классы нам будут необходимы для дальнейшей реализации модули внимания.

In [ ]:
import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, Layer, Input

from tensorflow.keras import backend as K

В качестве набора данных возьмем обработанный набор "Depression and Anxiety in Twitter (ID)" [(оригинал)](https://www.kaggle.com/stevenhans/depression-and-anxiety-in-twitter-id). Это набор твитов на индонезийском языке, часть из которых содержат депрессивные или тревожные мысли. В каждой строке данного набора два объекта - текст твита на индонезийском языке и бинарная метка наличия депрессивных мыслей в тексте. Таким образом, входными данными будет текст на естественном языке, решаемая задача - бинарная классификация.

In [ ]:
train = pd.read_csv('datd_train.csv')
test = pd.read_csv('datd_test.csv')
train.head()

,text,label
0,"oh pantesan tadi pada rame, ternyata monek mau...",0
1,"Semakin bertambah usia, semakin cemas hidup.",0
2,gelisah bgt astaga,1
3,Udah jangan terlalu cemas sikapku tak berubah ...,0
4,Giliran Aldebaran diambang kematian...Semua ba...,0


Разделим тренировочный и тестовые наборы на текст и разметку.

In [ ]:
X_train = train['text']
y_train = train['label'].tolist()

X_test = test['label']
y_test = test['label'].tolist()

print(X_train[0], " -- ", y_train[0])

oh pantesan tadi pada rame, ternyata monek mau cb juni. cemas aku tuh  --  0


Для использования текста в качестве тренировочных данных его необходимо предобработать. В данном случае мы проведем векторизацию текста - соотнесем каждое слово с определенным вектором фиксированного размера. Используя заранее известный размер словаря, создадим модуль векторизации и предварительно натренируем его на входном тексте.

In [ ]:
X_train.shape

(2201,)

In [ ]:
VOCAB_SIZE = 4000

encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(X_train)

2022-09-23 17:15:20.545154: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Далее создадим модель для классификации. Для обработки слоя сразу после энкодера, занимающегося векторизацией текста, добавим Embedding слой - данный слой будет получать на вход числовое представление текста после векторизации и преобразовывать в плотный вектор вещественных чисел. Это позволит сократить длину вектора и перейти от целочисленного представления (после векторизатора) к вещественному.  

Следующими идут два LSTM слоя, обернутых в директиву Bidirectional. Данная директива позволит слоям обработать текст дважды - слева-направо и справа-налево, позволяя учитывать контекст для каждого слова с обеих сторон. Первый слой из этих двух инициализирован параметром `return_sequences`, что позволит следующему слову получить все скрытые состояние предыдущего, а не только последнее. В итоге выходной вектор будет трехмерным: $[batch, timestep, data]$. Второй слой инициализирован без данного параметра, т.к. выходом для последующего полносвязного слоя должен служить двумерный вектор $[batch, data]$.

In [ ]:
model_bilstm = tf.keras.Sequential([
    Input(shape=(1,), dtype=tf.string),
    encoder,
    Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=100),
    Bidirectional(LSTM(32, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

Т.к. это задача бинарной классификации, скомпилируем модель с бинарной кроссэнтропией в качестве функции потерь и метрикой точности.

In [ ]:
model_bilstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Запустим процесс обучения. Заметим, каких значений достигла точность и функция потерь на валидационной выборке.

In [ ]:
history = model_bilstm.fit(train['text'], train['label'], validation_data=(test['text'], test['label']), epochs=10, batch_size=128)

Epoch 1/10


2022-09-23 17:15:23.563519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:15:23.941789: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:15:23.957201: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:15:24.110669: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:15:24.121986: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:15:24.300808: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:15:24.319074: I tensorflow/core/grappler/optimizers/cust

18/18 [==============================] - ETA: 0s - loss: 0.6504 - accuracy: 0.6533

2022-09-23 17:15:31.165715: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:15:31.299139: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:15:31.306596: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:15:31.427983: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:15:31.436094: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 [==============================] - 11s 438ms/step - loss: 0.6504 - accuracy: 0.6533 - val_loss: 0.5720 - val_accuracy: 0.7073
Epoch 2/10
18/18 [==============================] - 3s 192ms/step - loss: 0.5589 - accuracy: 0.7083 - val_loss: 0.4701 - val_accuracy: 0.7818
Epoch 3/10
18/18 [==============================] - 2s 132ms/step - loss: 0.4322 - accuracy: 0.8192 - val_loss: 0.4216 - val_accuracy: 0.8109
Epoch 4/10
18/18 [==============================] - 2s 122ms/step - loss: 0.3086 - accuracy: 0.8869 - val_loss: 0.4284 - val_accuracy: 0.8091
Epoch 5/10
18/18 [==============================] - 2s 124ms/step - loss: 0.2229 - accuracy: 0.9269 - val_loss: 0.5348 - val_accuracy: 0.7927
Epoch 6/10
18/18 [==============================] - 2s 119ms/step - loss: 0.1766 - accuracy: 0.9446 - val_loss: 0.5287 - val_accuracy: 0.7873
Epoch 7/10
18/18 [==============================] - 2s 116ms/step - loss: 0.1582 - accuracy: 0.9487 - val_loss: 0.5977 - val_accuracy: 0.8000
Epoch 8/10
18/18

Далее мы попробуем реализовать и добавить модуль внимания. В данном модуле мы будем учитывать принимать данные с предыдущего рекуррентного слоя, оценивать каждый из векторов с помощью полносвязного слоя, а затем складывать их с получившимися весами.  

Давайте реализуем данный модуль самостоятельно.

In [ ]:
class Attention(Layer):
    """
    Реализуем модуль внимания самостоятельно, используя возможности фреймворка TensorFlow. 
    В данном случае модуль будет принимать трехмерный тензор, осуществлять механизм внимания и
    возвращать суммированный двумерный вектор с учетом внимания.
    """
    
    def __init__(self, return_sequences=True):
        """
        На вход данного слоя следует подавать 3-хмерный тензор: [batch, time, data],
        поэтому все предыдущие LSTM слои должны быть инициализированы с "return_sequences=True"
        
        Если вы хотите использовать LSTM слои после данного слоя - инициализируйте этот слой с `return_sequences=True`,
        тогда выходом будет [batch, time, data]
        
        В противном случае данные с каждого шага времени будут суммированы и размерность выхода будет следующая: [batch, time]. 
        В таком случае следующем слоем должен быть слой, ожидающий двумерный вход (например, Dense)
        """
        
        self.return_sequences = return_sequences
        super().__init__()
        
    def build(self, input_shape):
        # Веса слоя
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        super().build(input_shape)
        
    def call(self, x):
        
        # Реализуем обычный полносвязный слой, умножив входящий тензор на веса, после чего применим нелинейную активацию
        e = K.tanh(K.dot(x, self.W))
        
        # Далее посчитаем softmax от полученных векторов по размерности time, т.о. присвоив важность каждому из векторов data
        a = K.softmax(e, axis=1)
        
        # Далее возьмем входные данные и домножим на получившиеся веса - каждый из векторов data будет домножен на соответствующий вес внимания
        output = x*a
        
        # Если в дальнейшем необходимо сохранить оригинальные данные, возвращаем трехмерный вектор
        if self.return_sequences:
            return output
        
        # Иначе - суммируем вектор по оси time, получая двумерный вектор. 
        # Таким образом, мы сложили все векторы data по оси time, домножив их на веса от механизма внимания.
        return K.sum(output, axis=1)

Создадим модель, аналогичную предыдущей, в которую добавим слои внимания. Учтите, что теперь оба LSTM слоя инициализированы с `return_sequence` для возвращения трехмерного вектора $[batch, timestep, data]$, а последний слой внимания инициалирован без данного параметра для корректной передачи данных далее в полносвязный слой.

In [ ]:
model_att = tf.keras.Sequential([
    Input(shape=(1,), dtype=tf.string),
    encoder,
    Embedding(input_dim=len(encoder.get_vocabulary()), output_dim=100),
    Bidirectional(LSTM(32, return_sequences=True)),
    Attention(return_sequences=True),
    Bidirectional(LSTM(32, return_sequences=True)),
    Attention(return_sequences=False),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
model_att.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Натренируем модель с механизмом внимания.

In [ ]:
history = model_att.fit(train['text'], train['label'], validation_data=(test['text'], test['label']), epochs=10, batch_size=128)

Epoch 1/10


2022-09-23 17:16:31.829242: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:16:32.218102: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:16:32.250071: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:16:32.595933: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:16:32.606394: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:16:33.030687: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:16:33.048797: I tensorflow/core/grappler/optimizers/cust

18/18 [==============================] - ETA: 0s - loss: 0.6784 - accuracy: 0.6470

2022-09-23 17:16:41.107126: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:16:41.254334: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:16:41.262632: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:16:41.408143: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-23 17:16:41.415478: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


18/18 [==============================] - 13s 511ms/step - loss: 0.6784 - accuracy: 0.6470 - val_loss: 0.6376 - val_accuracy: 0.7073
Epoch 2/10
18/18 [==============================] - 3s 177ms/step - loss: 0.6438 - accuracy: 0.6670 - val_loss: 0.6151 - val_accuracy: 0.7073
Epoch 3/10
18/18 [==============================] - 2s 139ms/step - loss: 0.6403 - accuracy: 0.6670 - val_loss: 0.6105 - val_accuracy: 0.7073
Epoch 4/10
18/18 [==============================] - 3s 165ms/step - loss: 0.6286 - accuracy: 0.6670 - val_loss: 0.5679 - val_accuracy: 0.7073
Epoch 5/10
18/18 [==============================] - 3s 143ms/step - loss: 0.5482 - accuracy: 0.6692 - val_loss: 0.4646 - val_accuracy: 0.7818
Epoch 6/10
18/18 [==============================] - 3s 145ms/step - loss: 0.4557 - accuracy: 0.8124 - val_loss: 0.4567 - val_accuracy: 0.8182
Epoch 7/10
18/18 [==============================] - 3s 159ms/step - loss: 0.3915 - accuracy: 0.8569 - val_loss: 0.4582 - val_accuracy: 0.8182
Epoch 8/10
18/18

Отметьте, каким стало значение функции потерь и точности на валидационной выборке.  

Отличаются ли они? Насколько сильно? Учитывайте, что данный набор данных достаточно прост для этой задачи. Каким образом это может влиять на эффективность механизма внимания?

Сможете ли вы изменить сеть или данные так, чтобы влияние механизма внимания стало более заметно?
